# Prize-Collecting Steiner Tree (PCSTP)

## Libs Importing

In [1]:
import sys
import os
import time
import networkx as nx

sys.path.insert(1, os.path.realpath(os.path.pardir))

In [2]:
import multiprocessing

NUM_PROCESSES = multiprocessing.cpu_count()
print("Number of cpu : ", NUM_PROCESSES)

Number of cpu :  12


In [3]:
from pcstp.instances.generator import generate_random_steiner
from pcstp.instances.reader import SteinlibReader, DatReader

from pcstp.steinertree import SteinerTreeProblem
from pcstp.solver.base import computes_steiner_cost
from pcstp.solver.aco import AntColony
from pcstp.solver.greedy_h1 import GreedyH1

from pcstp.utils.graph import preprocessing
from pcstp.utils.draw import draw_steiner_graph

## Experiments

In [4]:
SEED = 100

In [5]:
import glob
import numpy as np
import random

INSTANCES_PATH_PREFIX = '../data/instances/benchmark/SteinCD'
NUM_EXPERIMENTS_PER_INSTANCE = 10

all_files = glob.glob(os.path.join(INSTANCES_PATH_PREFIX, '*'))

files = all_files

networkx_history = []
solutions = {}
for filename in files:
    if filename.endswith('.xlsx') or filename.endswith('.csv'): continue
    if filename.endswith('.stp'):
        stp_reader = SteinlibReader()
    else:
        stp_reader = DatReader()

    print(f"Reading: {filename}")
    stp = stp_reader.parser(filename=filename)
    G, terminals = preprocessing(stp.graph, stp.terminals)
    stp_preprocessed = SteinerTreeProblem(graph=G, terminals=terminals)
    
    def run_experiment(experiment: int):
        if SEED:
            np.random.seed(SEED*experiment)
            random.seed(SEED*experiment)
        start_time = time.time()
        nx_steiner_tree = nx.algorithms.approximation.steiner_tree(
            stp_preprocessed.graph,
            stp_preprocessed.terminals,
            weight='cost'
        )

        networkx_duration = time.time() - start_time
        networkx_cost = computes_steiner_cost(stp.graph, nx_steiner_tree, stp.terminals)

        history = {
            "filename": filename,
            "experiment": experiment,
            "num_nodes": stp.num_nodes,
            "num_edges": stp.num_edges,
            "num_nodes_after_preprocessing": len(stp_preprocessed.graph.nodes),
            "num_edges_after_preprocessing": len(stp_preprocessed.graph.edges),
            "terminals": stp.num_terminals,
            "steiner_cost": networkx_cost,
            "duration": networkx_duration
        }
        print('Cost: ', networkx_cost)
        return history, nx_steiner_tree.copy()
        
    experiments = range(1, NUM_EXPERIMENTS_PER_INSTANCE+1)

    with multiprocessing.Pool(processes=NUM_PROCESSES) as p:
        experiments_results = p.map(run_experiment, experiments)
    
    networkx_history.extend([result[0] for result in experiments_results])
    solutions[filename] = [result[1] for result in experiments_results]

Reading: ../data/instances/benchmark/SteinCD/steinc6-wmax_100-seed_33000-gw.dat
Cost:  60.0
Cost:  60.0
Cost:  60.0
Cost:  60.0
Cost:  60.0
Cost:  60.0
Cost:  60.0
Cost:  60.0
Cost:  60.0
Cost:  60.0
Reading: ../data/instances/benchmark/SteinCD/steinc8-wmax_100-seed_33000-gw.dat
Cost:  530.0
Cost:  530.0Cost: 
 530.0
Cost:  530.0
Cost:  530.0
Cost:  530.0
Cost:  530.0
Cost:  Cost: 530.0
 530.0
Cost:  530.0
Reading: ../data/instances/benchmark/SteinCD/steinc7-wmax_100-seed_33000-gw.dat
Cost:  115.0
Cost:  Cost: 115.0 115.0

Cost: Cost:   115.0
115.0
Cost:  115.0
Cost:  115.0
Cost:  115.0
Cost:  115.0
Cost:  115.0
Reading: ../data/instances/benchmark/SteinCD/steinc9-wmax_100-seed_33000-gw.dat
Cost:  729.0
Cost: Cost:  729.0
 729.0
Cost:  729.0
Cost:  729.0
Cost:  729.0
Cost: Cost:  729.0
 729.0
Cost:  729.0
Cost:  729.0
Reading: ../data/instances/benchmark/SteinCD/steinc10-wmax_100-seed_33000-gw.dat
Cost:  1121.0
Cost:  Cost: 1121.0
 1121.0
Cost:  1121.0
Cost:  1121.0
Cost:  1121.0
Cost:

In [6]:
import pandas as pd

df_score_networkx = pd.DataFrame.from_dict(networkx_history)
df_score_networkx.to_csv(os.path.join(INSTANCES_PATH_PREFIX, 'NETWORKX.csv'))

In [7]:
df_score_networkx.groupby('filename')[['duration', 'steiner_cost']].describe()

duration            \
                                                      count      mean   
filename                                                                
../data/instances/benchmark/SteinCD/steinc10-wm...     10.0  5.610711   
../data/instances/benchmark/SteinCD/steinc6-wma...     10.0  4.929784   
../data/instances/benchmark/SteinCD/steinc7-wma...     10.0  4.461129   
../data/instances/benchmark/SteinCD/steinc8-wma...     10.0  5.254678   
../data/instances/benchmark/SteinCD/steinc9-wma...     10.0  5.305396   

                                                                        \
                                                         std       min   
filename                                                                 
../data/instances/benchmark/SteinCD/steinc10-wm...  1.072187  4.212903   
../data/instances/benchmark/SteinCD/steinc6-wma...  0.689333  3.915641   
../data/instances/benchmark/SteinCD/steinc7-wma...  0.548962  3.500526   
../data/instances/benchmark/SteinCD/steinc8-wma...  1.121896  3.698000   
../data/instances/benchmark/SteinCD/steinc9-wma...  1.146874  4.152395   

                                                                        \
                                                         25%       50%   
filename                                                                 
../data/instances/benchmark/SteinCD/steinc10-wm...  4.864818  5.223926   
../data/instances/benchmark/SteinCD/steinc6-wma...  4.394721  4.898269   
../data/instances/benchmark/SteinCD/steinc7-wma...  4.160000  4.441877   
../data/instances/benchmark/SteinCD/steinc8-wma...  4.201929  5.381670   
../data/instances/benchmark/SteinCD/steinc9-wma...  4.512299  4.914192   

                                                                        \
                                                         75%       max   
filename                                                                 
../data/instances/benchmark/SteinCD/steinc10-wm...  6.603621  7.226060   
../data/instances/benchmark/SteinCD/steinc6-wma...  5.514147  5.973273   
../data/instances/benchmark/SteinCD/steinc7-wma...  4.669987  5.605597   
../data/instances/benchmark/SteinCD/steinc8-wma...  6.310937  6.685112   
../data/instances/benchmark/SteinCD/steinc9-wma...  5.478586  7.639112   

                                                   steiner_cost               \
                                                          count    mean  std   
filename                                                                       
../data/instances/benchmark/SteinCD/steinc10-wm...         10.0  1121.0  0.0   
../data/instances/benchmark/SteinCD/steinc6-wma...         10.0    60.0  0.0   
../data/instances/benchmark/SteinCD/steinc7-wma...         10.0   115.0  0.0   
../data/instances/benchmark/SteinCD/steinc8-wma...         10.0   530.0  0.0   
../data/instances/benchmark/SteinCD/steinc9-wma...         10.0   729.0  0.0   

                                                                            \
                                                       min     25%     50%   
filename                                                                     
../data/instances/benchmark/SteinCD/steinc10-wm...  1121.0  1121.0  1121.0   
../data/instances/benchmark/SteinCD/steinc6-wma...    60.0    60.0    60.0   
../data/instances/benchmark/SteinCD/steinc7-wma...   115.0   115.0   115.0   
../data/instances/benchmark/SteinCD/steinc8-wma...   530.0   530.0   530.0   
../data/instances/benchmark/SteinCD/steinc9-wma...   729.0   729.0   729.0   

                                                                    
                                                       75%     max  
filename                                                            
../data/instances/benchmark/SteinCD/steinc10-wm...  1121.0  1121.0  
../data/instances/benchmark/SteinCD/steinc6-wma...    60.0    60.0  
../data/instances/benchmark/SteinCD/steinc7-wma...   115.0   115.0  
../data

In [8]:
# experiment = 4
# G = solutions['../data/instances/benchmark/PCSPG-CRR/C03-A.stp'][experiment].graph
# steiner_tree = solutions['../data/instances/benchmark/PCSPG-CRR/C03-A.stp'][experiment]

KeyError: '../data/instances/benchmark/PCSPG-CRR/C03-A.stp'

In [10]:
# steiner_tree